In [1]:
from torch_geometric.datasets import Planetoid
import torch
from torch_geometric.data import Data
from torch_geometric.data import DataLoader

In [2]:
x = torch.tensor([[-1, 0, 1], [0, 1, 2]], dtype=torch.float)
y = torch.tensor([0, 1,3,4,5,6,], dtype=torch.long)
z = torch.tensor([0, 0,3,4,5,0,], dtype=torch.long)

In [3]:
non_zero_count = torch.nonzero(y).size(0)
non_zero_count

5

In [4]:
mask_nonzero_y = (y != 0)
mask_nonzero_z = (z != 0)
mask_nonzero_different = (mask_nonzero_y & mask_nonzero_z & (y == z))

# 计算满足条件的元素数量
count_nonzero_different = torch.sum(mask_nonzero_different).item()
count_nonzero_different

3

In [5]:
data = Data(x=x, edge_index=edge_index, y=y)
data.num_nodes

2

In [3]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [4]:
custom_dataset = CustomDataset(dataset_list)
custom_dataset[0]

Data(x=[2, 3], edge_index=[2, 2], y=[2])

In [5]:
dataloader = DataLoader(custom_dataset, batch_size=1, shuffle=True)

/home/btr/miniconda3/envs/LLMEnG/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [6]:
for batch in dataloader:
    print(batch.x)
    print(batch.y)

tensor([[-1.,  0.,  1.],
        [ 0.,  1.,  2.]])
tensor([0, 1])


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = SAGEConv(3, 3, aggr='mean')
        self.conv2 = SAGEConv(3, 2, aggr='mean')

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        print(x)
        # x = F.relu(x)
        # x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return x

In [14]:
model = Net()
optizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

model.train()
for epoch in range(10):
    for data in dataloader:
        optizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y)
        unique_batch_indices = torch.unique(data.batch)
        print(unique_batch_indices)
        # 迭代并处理每个图
        loss.backward()
        optizer.step()

tensor([[ 0.6433,  0.2546,  0.2448],
        [ 0.4562,  1.4739, -0.6014]], grad_fn=<AddBackward0>)
tensor([0])
Data(x=[2, 3], edge_index=[2, 2], y=[2])
子图 0 的节点特征：
tensor([[-1.,  0.,  1.],
        [ 0.,  1.,  2.]])

tensor([[ 0.6372,  0.2498,  0.2353],
        [ 0.4631,  1.4760, -0.5987]], grad_fn=<AddBackward0>)
tensor([0])
Data(x=[2, 3], edge_index=[2, 2], y=[2])
子图 0 的节点特征：
tensor([[-1.,  0.,  1.],
        [ 0.,  1.,  2.]])

tensor([[ 0.6311,  0.2450,  0.2260],
        [ 0.4699,  1.4782, -0.5960]], grad_fn=<AddBackward0>)
tensor([0])
Data(x=[2, 3], edge_index=[2, 2], y=[2])
子图 0 的节点特征：
tensor([[-1.,  0.,  1.],
        [ 0.,  1.,  2.]])

tensor([[ 0.6251,  0.2402,  0.2169],
        [ 0.4766,  1.4806, -0.5934]], grad_fn=<AddBackward0>)
tensor([0])
Data(x=[2, 3], edge_index=[2, 2], y=[2])
子图 0 的节点特征：
tensor([[-1.,  0.,  1.],
        [ 0.,  1.,  2.]])

tensor([[ 0.6192,  0.2353,  0.2080],
        [ 0.4833,  1.4830, -0.5909]], grad_fn=<AddBackward0>)
tensor([0])
Data(x=[2, 3], edge_index

In [15]:
import json
import numpy as np
import random
def __generate_edge_index(data_2_mask_single_signal_llm:list[str]) -> list[list[int, int]]:
    '''
        生成邻接矩阵
        data_2_mask_single_signal_llm: list[str], 数据集数据
    '''
    cur_tokens = data_2_mask_single_signal_llm
    cur_no_activity = []
    last_activity = -1
    cur_adj_lists = []
    for j in range(0, len(cur_tokens)):
        if cur_tokens[j] != '[activity]':
            cur_no_activity.append(j)
            if last_activity != -1:
                cur_adj_lists.append([last_activity, j])
                cur_adj_lists.append([j, last_activity])
                # cur_adj_lists[last_activity].add(j)
                # cur_adj_lists[j].add(last_activity)
        elif cur_tokens[j] == '[activity]' and len(cur_no_activity) != 0:
            for k in cur_no_activity:
                cur_adj_lists.append([k, j])
                cur_adj_lists.append([j, k])
                # cur_adj_lists[k].add(j)
                # cur_adj_lists[j].add(k)
            last_activity = j
            cur_no_activity = []
        elif cur_tokens[j] == '[activity]' and len(cur_no_activity) == 0:
            #两个activity直接相连的时候，或许可以考虑在中间加个什么标识
            if last_activity != -1:
                cur_adj_lists.append([last_activity, j])
                cur_adj_lists.append([j, last_activity])
                # cur_adj_lists[last_activity].add(j)
                # cur_adj_lists[j].add(last_activity)
            last_activity = j
    cur_adj_lists = sorted(cur_adj_lists, key=lambda x: x[1])
    return cur_adj_lists

def __generate_edge_index_2(relation_matrix:list[list[int]]) -> list[list[int]]:
    '''
        生成邻接矩阵
        relation_matrix: list[list[int]], 邻接矩阵
    '''
    edge_index = []
    for i in range(0, len(relation_matrix)):
        for j in range(0, len(relation_matrix[i])):
            if relation_matrix[i][j] == 1:
                edge_index.append([i, j])
    edge_index = sorted(edge_index, key=lambda x: x[1])
    return edge_index
def __modify_edge_index(orginal_edge_index:list[list[int]], llm_edge_index:list[list[int]]) -> list[list[int]]:
    if llm_edge_index == []:
        edge_index = orginal_edge_index
        edge_index = sorted(edge_index, key=lambda x: x[1])
        return edge_index
    orginal_edge_index = np.array(orginal_edge_index)
    llm_edge_index = np.array(llm_edge_index)
    edge_index = np.unique(np.vstack((orginal_edge_index, llm_edge_index)), axis=0).tolist()
    # print(edge_index)
    # print(orginal_edge_index.tolist())
    edge_index = sorted(edge_index, key=lambda x: x[1])
    return edge_index

def __modify_edge_index_2(orginal_edge_index:list[list[int]], llm_edge_index:list[list[int]]) -> list[list[int]]:
    temp = [node_pair for node_pair in llm_edge_index if node_pair not in orginal_edge_index]
    num = int(len(orginal_edge_index) * 0.2)
    edge_index = []
    if len(temp) <= num:
        edge_index = orginal_edge_index + edge_index
    else:
        random_elements = random.sample(temp, num)
        edge_index = orginal_edge_index + random_elements
    return edge_index

dataset_list = None
with open('/home/btr/bpmn/LLMEnG/my-data/datasets8-llama3.1-8B.json', 'r') as f:
    dataset_list = json.load(f)
for dataset in dataset_list:
    data_2_mask_single_signal_llm = dataset['data_2_mask_single_signal_llm']
    relation_matrix = dataset['relation_matrix']
    orginal_edge_index = __generate_edge_index(data_2_mask_single_signal_llm)
    llm_edge_index = __generate_edge_index_2(relation_matrix)
    edge_index = __modify_edge_index_2(orginal_edge_index, llm_edge_index)
    print(len(orginal_edge_index), len(llm_edge_index), len(edge_index))
    
    


52 84 62
40 10 48
102 64 122
18 4 18
96 264 115
34 20 40
62 70 74
50 8 50
52 76 62
24 8 24
54 26 64
38 32 45
24 12 28
30 24 36
38 66 45
60 72 72
70 134 84
46 126 55
48 102 57
32 12 38
42 92 50
40 6 40
54 44 64
42 40 50
30 42 36
50 114 60
12 4 12
40 20 48
62 132 74
34 98 40
20 10 24
202 1298 242
136 208 163
28 26 33
26 8 26
22 22 26
34 26 40
44 56 52
102 88 122
28 14 28
28 32 33
58 230 69
12 2 12
38 50 45
72 46 86
24 6 28
14 6 16
32 60 38
26 22 31
64 116 76
8 0 8
16 2 16
54 90 64
88 298 105
8 2 8
32 0 32
20 6 20
6 0 6
8 0 8
12 4 12
16 0 16
20 14 24
28 14 33
18 12 21
10 14 12
60 44 72
50 28 60
56 42 67


In [7]:
a = [[1,2], [2, 3]]
b = [[1,2], [5, 6], [1,43]]

# 将子列表转换为元组以便于使用集合
print(a+b)

[[1, 2], [2, 3], [1, 2], [5, 6], [1, 43]]


In [13]:
import random

# 定义列表a和b
a = [[1, 2], [2, 3]]
b = [[1, 2], [5, 6], [1, 43]]

# 步骤1: 找出出现在b中但不出现在a中的元素，加入到临时列表temp中
temp = [item for item in b if item not in a]

# 输出临时列表temp
print("临时列表 temp:", temp)

# 步骤2: 计算a列表的长度并乘以0.2，向下取整，得到整数num
num = int(len(a) * 0.2)

# 输出计算得到的num
print("计算得到的num:", num)

# 步骤3: 判断b的长度与num的大小关系
if len(b) <= num:
    # 如果b的长度小于等于num，合并a和b
    c = a + b
else:
    # 如果b的长度大于num，在b中随机取num个元素
    random_elements = random.sample(b, num)
    # 合并a和随机取出的b元素
    c = a + random_elements

# 输出最终结果c
print("最终列表 c:", c)

临时列表 temp: [[5, 6], [1, 43]]
计算得到的num: 0
最终列表 c: [[1, 2], [2, 3]]


In [39]:
import torch
from torch import nn
import math
 
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(in_channels=1,out_channels=1,kernel_size=2,stride=1,padding=0)   
    def forward(self,x):
        out = self.conv(x)
        return out
 
net = Net()

optimizer = torch.optim.SGD(net.parameters(), lr = 0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.95)

for i in range(150):
    print("lr of epoch", i, "=>", scheduler.get_lr())
    optimizer.step()
    scheduler.step()


lr of epoch 0 => [0.01]
lr of epoch 1 => [0.01]
lr of epoch 2 => [0.01]
lr of epoch 3 => [0.01]
lr of epoch 4 => [0.01]
lr of epoch 5 => [0.01]
lr of epoch 6 => [0.01]
lr of epoch 7 => [0.01]
lr of epoch 8 => [0.01]
lr of epoch 9 => [0.01]
lr of epoch 10 => [0.01]
lr of epoch 11 => [0.01]
lr of epoch 12 => [0.01]
lr of epoch 13 => [0.01]
lr of epoch 14 => [0.01]
lr of epoch 15 => [0.01]
lr of epoch 16 => [0.01]
lr of epoch 17 => [0.01]
lr of epoch 18 => [0.01]
lr of epoch 19 => [0.01]
lr of epoch 20 => [0.01]
lr of epoch 21 => [0.01]
lr of epoch 22 => [0.01]
lr of epoch 23 => [0.01]
lr of epoch 24 => [0.01]
lr of epoch 25 => [0.01]
lr of epoch 26 => [0.01]
lr of epoch 27 => [0.01]
lr of epoch 28 => [0.01]
lr of epoch 29 => [0.01]
lr of epoch 30 => [0.01]
lr of epoch 31 => [0.01]
lr of epoch 32 => [0.01]
lr of epoch 33 => [0.01]
lr of epoch 34 => [0.01]
lr of epoch 35 => [0.01]
lr of epoch 36 => [0.01]
lr of epoch 37 => [0.01]
lr of epoch 38 => [0.01]
lr of epoch 39 => [0.01]
lr of epoc